In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [2]:
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIIZE_STD = [0.229, 0.224, 0.225]

IMAGE_SIZE = 224
BATCH_SIZE = 16

NUM_EPOCHS = 30
NUM_CLASSES = 9
NUM_WORKERS = 2
LEARNING_RATE = 1e-3

In [3]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIIZE_STD),
])

val_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIIZE_STD),
])

train_dataset_path = "../../../dataset/train"
val_dataset_path = "../../../dataset/val"

train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
val_dataset = datasets.ImageFolder(root=val_dataset_path, transform=val_transforms)

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

val_loader = DataLoader(
    val_dataset,
    shuffle=True,
    pin_memory=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [4]:
if __name__ == "__main__":
    images, labels = next(iter(train_loader))
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {images.shape}")

Batch shape: torch.Size([16, 3, 224, 224])
Labels shape: torch.Size([16, 3, 224, 224])


In [5]:
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)

In [7]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    training_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        training_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
    
    avg_loss = training_loss/total
    avg_accuracy = correct/total

    return avg_loss, avg_accuracy

In [8]:
def validate(model, loader, criterion, device):
    model.eval()
    validate_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            validate_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    
    avg_loss = validate_loss/total 
    avg_accuracy = correct/total 

    return avg_loss, avg_accuracy

In [9]:
best_val_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    validate_loss, val_acc = validate(model, train_loader, criterion, device)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: "
          f"Train loss {train_loss:.4f} acc {train_acc:.4f} | "
          f"Val   loss {validate_loss:.4f} acc {val_acc:.4f}")
    
    # checkpoint best model
    if validate_loss < best_val_loss:
        best_val_loss = validate_loss
        torch.save(model.state_dict(), "best_resnet152.pth")


Epoch 1/30: Train loss 1.0124 acc 0.6677 | Val   loss 0.5984 acc 0.8141
Epoch 2/30: Train loss 0.6492 acc 0.7748 | Val   loss 0.6097 acc 0.7899
Epoch 3/30: Train loss 0.5679 acc 0.8042 | Val   loss 0.4779 acc 0.8343
Epoch 4/30: Train loss 0.5079 acc 0.8184 | Val   loss 0.4258 acc 0.8687
Epoch 5/30: Train loss 0.5325 acc 0.8137 | Val   loss 0.3928 acc 0.8687
Epoch 6/30: Train loss 0.5071 acc 0.8147 | Val   loss 0.4260 acc 0.8545
Epoch 7/30: Train loss 0.4864 acc 0.8297 | Val   loss 0.3596 acc 0.8879
Epoch 8/30: Train loss 0.5041 acc 0.8201 | Val   loss 0.3123 acc 0.8990
Epoch 9/30: Train loss 0.4869 acc 0.8191 | Val   loss 0.3322 acc 0.8939
Epoch 10/30: Train loss 0.4718 acc 0.8359 | Val   loss 0.3878 acc 0.8768
Epoch 11/30: Train loss 0.4783 acc 0.8340 | Val   loss 0.3212 acc 0.8949
Epoch 12/30: Train loss 0.4752 acc 0.8305 | Val   loss 0.3950 acc 0.8616
Epoch 13/30: Train loss 0.4557 acc 0.8366 | Val   loss 0.3995 acc 0.8576
Epoch 14/30: Train loss 0.4855 acc 0.8279 | Val   loss 0.301